In [21]:
import requests
import pyodbc as db
import os
import time

api_key = '3aa6712010d64ae6a03f02d61f23bf77' #Esta clave se genera al crear un usuario 

In [56]:
def get_connection():
   # Conexión a la base de datos
   return db.connect(
                    f"Driver={{ODBC Driver 17 for SQL Server}};"
                    f"Server={os.getenv('DB_SERVER')};"
                    f"Database={os.getenv('DB_NAME')};"
                    f"UID={os.getenv('DB_USER')};"
                    f"PWD={os.getenv('DB_PASS')};"
                    "Trusted_Connection=no;"
                    "Integrated_Security=false;"
                    )

In [57]:
def get_location (conn):
    cursor = conn.cursor()
    cursor.execute("SELECT distinct location from Timezones")
    location = [row[0] for row in cursor.fetchall()]
    
    location = [x for x in location if x is not None]  #Este paso está aqui sólo para eliminar nulos si los hubiese
    return location

In [ ]:
def update_utc_offset (conn, locations):
    cursor = conn.cursor()

    for location in locations:
        try:
            response = requests.get(f"https://timezone.abstractapi.com/v1/current_time/?api_key={api_key}&location={location}")
            result = response.json()
            utc_diff = result["gmt_offset"]
            
            actual_dif = cursor.execute(f"SELECT dif from Timezones where location = '{location}'")
            actual_dif = actual_dif.fetchone()[0]

            if actual_dif != utc_diff:
                cursor.execute(f''' UPDATE Timezones
                                    SET dif = {utc_diff}
                                    WHERE location = '{location}'
                                ''')
                
                cursor.commit()
                print(f'''Se ha actualizado el offset_UTC para los partner con location: {location}.
                          offset_UTC anterior: {actual_dif} 
                          offset_UTC nuevo: {utc_diff} 
                      ''')
                time.sleep(1)  # Espera 1 segundo para asegurar no incumplir con las normas de la api de 1 solicitud x segundo máx.

        except Exception as e:
            print (f"""Error al intentar actualizar el offset UTC de la timezone: {location}.
                       Type Error: {e}""")
    
    cursor.close()

In [ ]:
connect = get_connection()
locations = get_location (connect)

In [75]:
update_utc_offset (connect, locations)

location: Asuncion, Paraguay, actual_dif: -4, utc_diff: -4
location: Bogota, Colombia, actual_dif: -5, utc_diff: -5
location: Buenos Aires, Argentina, actual_dif: -3, utc_diff: -3
location: Costa_Rica, Costa Rica, actual_dif: -6, utc_diff: -6
location: El_Salvador, El Salvador, actual_dif: -22, utc_diff: -6
Se ha actualizado el offset_UTC para los partner con location: El_Salvador, El Salvador.
                          offset_UTC anterior: -22 
                          offset_UTC nuevo: -6 
                      
location: Guatemala, Guatemala, actual_dif: -6, utc_diff: -6
location: Guayaquil, Ecuador, actual_dif: -5, utc_diff: -5
location: La_Paz, Bolivia, actual_dif: -4, utc_diff: -4
location: Lima, Peru, actual_dif: -5, utc_diff: -5
location: Managua, Nicaragua, actual_dif: -6, utc_diff: -6
location: Mexico_city, Mexico, actual_dif: -6, utc_diff: -6
location: Panama, Panama, actual_dif: -5, utc_diff: -5
location: Santiago, Chile, actual_dif: -4, utc_diff: -4
location: Santo_Doming

In [76]:
connect.close()